In [1]:
import pandas as pd
import numpy as np

In [2]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
%matplotlib inline

In [3]:
df = pd.read_csv('project_data.csv')

In [4]:
df

,XVPM,GWYH,TRAT,TLLZ,IGGA,HYKR,EDFS,GUUB,MGJM,JHZC,TARGET CLASS
0,1636.670614,817.988525,2565.995189,358.347163,550.417491,1618.870897,2147.641254,330.727893,1494.878631,845.136088,0
1,1013.402760,577.587332,2644.141273,280.428203,1161.873391,2084.107872,853.404981,447.157619,1193.032521,861.081809,1
2,1300.035501,820.518697,2025.854469,525.562292,922.206261,2552.355407,818.676686,845.491492,1968.367513,1647.186291,1
3,1059.347542,1066.866418,612.000041,480.827789,419.467495,685.666983,852.867810,341.664784,1154.391368,1450.935357,0
4,1018.340526,1313.679056,950.622661,724.742174,843.065903,1370.554164,905.469453,658.118202,539.459350,1899.850792,0
...,...,...,...,...,...,...,...,...,...,...,...
995,1343.060600,1289.142057,407.307449,567.564764,1000.953905,919.602401,485.269059,668.007397,1124.772996,2127.628290,0
996,938.847057,1142.884331,2096.064295,483.242220,522.755771,1703.169782,2007.548635,533.514816,379.264597,567.200545,1
997,921.994822,607.996901,2065.482529,497.107790,457.430427,1577.506205,1659.197738,186.854577,978.340107,1943.304912,1
998,1157.069348,602.749160,1548.809995,646.809528,1335.737820,1455.504390,2788.366441,552.388107,1264.818079,1331.879020,1


In [5]:
data_core = df.drop('TARGET CLASS', axis=1).copy()
target = df['TARGET CLASS'].copy()

In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
scaler = StandardScaler()
standard_df = scaler.fit_transform(data_core)
standard_df = pd.DataFrame(standard_df, columns=data_core.columns)
standard_df.head()

,XVPM,GWYH,TRAT,TLLZ,IGGA,HYKR,EDFS,GUUB,MGJM,JHZC
0,1.568522,-0.443435,1.619808,-0.958255,-1.128481,0.138336,0.980493,-0.932794,1.008313,-1.069627
1,-0.112376,-1.056574,1.741918,-1.504220,0.640009,1.081552,-1.182663,-0.461864,0.258321,-1.041546
2,0.660647,-0.436981,0.775793,0.213394,-0.053171,2.030872,-1.240707,1.149298,2.184784,0.342811
3,0.011533,0.191324,-1.433473,-0.100053,-1.507223,-1.753632,-1.183561,-0.888557,0.162310,-0.002793
4,-0.099059,0.820815,-0.904346,1.609015,-0.282065,-0.365099,-1.095644,0.391419,-1.365603,0.787762


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [10]:
from sklearn.model_selection import GridSearchCV

In [11]:
from sklearn.metrics import confusion_matrix, classification_report

In [12]:
X = standard_df
y = target

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Gridsearch fo KNN

In [133]:
# Number of neighbors to use by default for kneighbors queries
n_neighbors = [x for x in range(1,151,2)]
# Weight function used in prediction
weights = ['uniform','distance']
# Algorithm used to compute the nearest neighbors
algorithm = ['ball_tree', 'kd_tree', 'brute']
p = [1, 2]

# Searching params for KNN classificator
param_knn = {'n_neighbors':n_neighbors, 'weights':weights, 'algorithm':algorithm, 'p':p}

In [134]:
knn_grid = GridSearchCV(KNeighborsClassifier(), param_knn, verbose=3, n_jobs=-1)
knn_grid.fit(X_train, y_train)

Fitting 5 folds for each of 900 candidates, totalling 4500 fits


GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['ball_tree', 'kd_tree', 'brute'],
                         'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21,
                                         23, 25, 27, 29, 31, 33, 35, 37, 39, 41,
                                         43, 45, 47, 49, 51, 53, 55, 57, 59, ...],
                         'p': [1, 2], 'weights': ['uniform', 'distance']},
             verbose=3)

In [135]:
print(knn_grid.best_score_, '\n', knn_grid.best_estimator_)

0.8371428571428572 
 KNeighborsClassifier(algorithm='ball_tree', n_neighbors=53, weights='distance')


In [136]:
knn_pred = knn_grid.predict(X_test)

In [137]:
# KNN classificator report
print(confusion_matrix(y_test, knn_pred),'\n',classification_report(y_test,knn_pred))

[[122  24]
 [ 27 127]] 
               precision    recall  f1-score   support

           0       0.82      0.84      0.83       146
           1       0.84      0.82      0.83       154

    accuracy                           0.83       300
   macro avg       0.83      0.83      0.83       300
weighted avg       0.83      0.83      0.83       300



# Gridsearch for SVC

In [28]:
# Regularization parameter
c = [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 5.0, 10.0, 15.0, 20.0, 25.0]
# Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’
gamma = [1, 0.1, 0.01, 0.001, 0.0001]
# Specifies the kernel type to be used in the algorithm
kernel = ['linear', 'poly', 'rbf', 'sigmoid']

# Searching params for SVC classificator
param_svc = {'C':c, 'gamma':gamma, 'kernel':kernel}
param_svc

{'C': [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 5.0, 10.0, 15.0, 20.0, 25.0],
 'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}

In [29]:
svc_grid = GridSearchCV(SVC(), param_svc, verbose=3, n_jobs=-1)
svc_grid.fit(X_train, y_train)

Fitting 5 folds for each of 220 candidates, totalling 1100 fits


GridSearchCV(estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 5.0, 10.0,
                               15.0, 20.0, 25.0],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             verbose=3)

In [30]:
print(svc_grid.best_score_, '\n', svc_grid.best_estimator_)

0.8414285714285714 
 SVC(C=15.0, gamma=0.001, kernel='sigmoid')


In [31]:
# SVC classificator report
svc_pred = svc_grid.predict(X_test)
print(confusion_matrix(y_test, svc_pred),'\n',classification_report(y_test, svc_pred))

[[130  16]
 [ 28 126]] 
               precision    recall  f1-score   support

           0       0.82      0.89      0.86       146
           1       0.89      0.82      0.85       154

    accuracy                           0.85       300
   macro avg       0.86      0.85      0.85       300
weighted avg       0.86      0.85      0.85       300



# Searching params for RF classificator

In [65]:
# Whether bootstrap samples are used when building trees
bootstrap = [True, False]
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(200,2000,5)]
# Number of features consider at evry split
max_features = ['auto', 'sqrt', 'log2']
# Maximum number of levels in trees
max_depth = [int(x) for x in np.linspace(10,1000,5)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 6, 8]

param_rf = { 'max_depth': max_depth,
             'max_features': max_features,
             'min_samples_leaf': min_samples_leaf,
             'min_samples_split': min_samples_split,
             'n_estimators': n_estimators
             }
param_rf

{'max_depth': [10, 257, 505, 752, 1000],
 'max_features': ['auto', 'sqrt', 'log2'],
 'min_samples_leaf': [1, 2, 4, 6, 8],
 'min_samples_split': [2, 5, 10, 14],
 'n_estimators': [200, 650, 1100, 1550, 2000]}

In [61]:
from tqdm.notebook import tqdm

In [68]:
rf_grid = GridSearchCV(RandomForestClassifier(), param_rf, verbose=75, n_jobs=-1)
rf_grid.fit(X_train, y_train)

Fitting 5 folds for each of 1500 candidates, totalling 7500 fits


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [10, 257, 505, 752, 1000],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4, 6, 8],
                         'min_samples_split': [2, 5, 10, 14],
                         'n_estimators': [200, 650, 1100, 1550, 2000]},
             verbose=75)

In [69]:
print(rf_grid.best_score_,'\n', rf_grid.best_estimator_)

0.82 
 RandomForestClassifier(max_depth=752, max_features='log2', min_samples_split=14,
                       n_estimators=200)


In [70]:
# SVC classificator report
rf_pred = rf_grid.predict(X_test)
print(confusion_matrix(y_test, svc_pred),'\n',classification_report(y_test, svc_pred))

[[130  16]
 [ 26 128]] 
               precision    recall  f1-score   support

           0       0.83      0.89      0.86       146
           1       0.89      0.83      0.86       154

    accuracy                           0.86       300
   macro avg       0.86      0.86      0.86       300
weighted avg       0.86      0.86      0.86       300



# Some  Hyperparameters optimization with Genetic Algorithm - Teapot

## TPOT + KNN

In [16]:
from tpot import TPOTClassifier

C:\ProgramData\Anaconda3\lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [129]:
# Number of neighbors to use by default for kneighbors queries
n_neighbors = [x for x in range(1, 151,2)]
# Weight function used in prediction
weights = ['uniform','distance']
# Algorithm used to compute the nearest neighbors
algorithm = ['ball_tree', 'kd_tree', 'brute']
p = [1, 2]

# Searching params for KNN classificator
param_knn = {'n_neighbors':n_neighbors, 'weights':weights, 'algorithm':algorithm, 'p':p}

In [146]:
tpot_classifier = TPOTClassifier(generations=5, population_size=24, offspring_size=12,
                                verbosity=2, early_stop=12,
                                config_dict={'sklearn.neighbors.KNeighborsClassifier': param_knn},
                                cv=4, scoring='accuracy', n_jobs=-1)
tpot_classifier.fit(X_train,y_train)

Optimization Progress:   0%|          | 0/84 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.8414285714285714

Generation 2 - Current best internal CV score: 0.8414285714285714

Generation 3 - Current best internal CV score: 0.8414285714285714

Generation 4 - Current best internal CV score: 0.8414285714285714

Generation 5 - Current best internal CV score: 0.8414285714285714

Best pipeline: KNeighborsClassifier(input_matrix, algorithm=ball_tree, n_neighbors=107, p=2, weights=uniform)


TPOTClassifier(config_dict={'sklearn.neighbors.KNeighborsClassifier': {'algorithm': ['ball_tree',
                                                                                     'kd_tree',
                                                                                     'brute'],
                                                                       'n_neighbors': [1,
                                                                                       3,
                                                                                       5,
                                                                                       7,
                                                                                       9,
                                                                                       11,
                                                                                       13,
                                                                               

In [147]:
tpot_predict = tpot_classifier.predict(X_test)

In [148]:
# KNN classificator report
print(confusion_matrix(y_test, tpot_predict),'\n',classification_report(y_test,tpot_predict))

[[121  25]
 [ 18 136]] 
               precision    recall  f1-score   support

           0       0.87      0.83      0.85       146
           1       0.84      0.88      0.86       154

    accuracy                           0.86       300
   macro avg       0.86      0.86      0.86       300
weighted avg       0.86      0.86      0.86       300



## TPOT + SVC

In [24]:
# Regularization parameter
c = [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 5.0, 10.0, 15.0, 20.0, 25.0]
# Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’
gamma = [1, 0.1, 0.01, 0.001, 0.0001]
# Specifies the kernel type to be used in the algorithm
kernel = ['linear', 'poly', 'rbf', 'sigmoid']

# Searching params for SVC classificator
param_svc = {'C':c, 'gamma':gamma, 'kernel':kernel}

In [25]:
tpot_classifier = TPOTClassifier(generations=5, population_size=24, offspring_size=12,
                                verbosity=2, early_stop=12,
                                config_dict={'sklearn.svm.SVC': param_svc},
                                cv=4, scoring='accuracy', n_jobs=-1)
tpot_classifier.fit(X_train,y_train)

Optimization Progress:   0%|          | 0/84 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.8342857142857143

Generation 2 - Current best internal CV score: 0.8357142857142857

Generation 3 - Current best internal CV score: 0.8357142857142857

Generation 4 - Current best internal CV score: 0.8357142857142857

Generation 5 - Current best internal CV score: 0.8357142857142857

Best pipeline: SVC(input_matrix, C=0.5, gamma=0.01, kernel=rbf)


TPOTClassifier(config_dict={'sklearn.svm.SVC': {'C': [0.0001, 0.001, 0.01, 0.1,
                                                      0.5, 1.0, 5.0, 10.0, 15.0,
                                                      20.0, 25.0],
                                                'gamma': [1, 0.1, 0.01, 0.001,
                                                          0.0001],
                                                'kernel': ['linear', 'poly',
                                                           'rbf', 'sigmoid']}},
               cv=4, early_stop=12, generations=5, n_jobs=-1, offspring_size=12,
               population_size=24, scoring='accuracy', verbosity=2)

In [26]:
tpot_predict = tpot_classifier.predict(X_test)

In [27]:
# SVC classificator report
print(confusion_matrix(y_test, tpot_predict),'\n',classification_report(y_test,tpot_predict))

[[130  16]
 [ 28 126]] 
               precision    recall  f1-score   support

           0       0.82      0.89      0.86       146
           1       0.89      0.82      0.85       154

    accuracy                           0.85       300
   macro avg       0.86      0.85      0.85       300
weighted avg       0.86      0.85      0.85       300



## TPOT + Random Forest

In [14]:
# Whether bootstrap samples are used when building trees
bootstrap = [True, False]
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(200,2000,5)]
# Number of features consider at evry split
max_features = ['auto', 'sqrt', 'log2']
# Maximum number of levels in trees
max_depth = [int(x) for x in np.linspace(10,1000,5)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 6, 8]

param_rf = { 'max_depth': max_depth,
             'max_features': max_features,
             'min_samples_leaf': min_samples_leaf,
             'min_samples_split': min_samples_split,
             'n_estimators': n_estimators
             }
param_rf

{'max_depth': [10, 257, 505, 752, 1000],
 'max_features': ['auto', 'sqrt', 'log2'],
 'min_samples_leaf': [1, 2, 4, 6, 8],
 'min_samples_split': [2, 5, 10, 14],
 'n_estimators': [200, 650, 1100, 1550, 2000]}

In [17]:
tpot_classifier = TPOTClassifier(generations=5, population_size=24, offspring_size=12,
                                verbosity=2, early_stop=12,
                                config_dict={'sklearn.ensemble.RandomForestClassifier': param_rf},
                                cv=4, scoring='accuracy', n_jobs=1)
tpot_classifier.fit(X_train,y_train)

Optimization Progress:   0%|          | 0/84 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.8214285714285714

Generation 2 - Current best internal CV score: 0.8214285714285714

Generation 3 - Current best internal CV score: 0.8214285714285714

Generation 4 - Current best internal CV score: 0.8214285714285714

Generation 5 - Current best internal CV score: 0.8214285714285714

Best pipeline: RandomForestClassifier(RandomForestClassifier(input_matrix, max_depth=505, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1100), max_depth=257, max_features=auto, min_samples_leaf=8, min_samples_split=5, n_estimators=1100)


TPOTClassifier(config_dict={'sklearn.ensemble.RandomForestClassifier': {'max_depth': [10,
                                                                                      257,
                                                                                      505,
                                                                                      752,
                                                                                      1000],
                                                                        'max_features': ['auto',
                                                                                         'sqrt',
                                                                                         'log2'],
                                                                        'min_samples_leaf': [1,
                                                                                             2,
                                                            

In [18]:
tpot_predict = tpot_classifier.predict(X_test)

In [19]:
# RF classificator report
print(confusion_matrix(y_test, tpot_predict),'\n',classification_report(y_test,tpot_predict))

[[124  22]
 [ 31 123]] 
               precision    recall  f1-score   support

           0       0.80      0.85      0.82       146
           1       0.85      0.80      0.82       154

    accuracy                           0.82       300
   macro avg       0.82      0.82      0.82       300
weighted avg       0.82      0.82      0.82       300

